# Stereo vision

## Preliminaries

The following image shows the main concept underlying stereo vision, i.e., triangulation.

![Stereo vision setup.](stereo_setup.png)

<center><small>Basic concept behind stereo vision - drawing inspired by the drawing in the [lecture by Robert Collins](http://www.cse.psu.edu/~rtc12/CSE486/lecture09.pdf)</small></center>

The left camera is situated at the world coordinate $(0,0,0)$, while the right camera is situated at $(T_X, 0, 0)$. Both cameras observe points in the world, such as the point represented by a red star, situated at the world coordinate $(X,Y,Z)$. The figure shows the image planes in front of the camera, and the assumption is made that they are perfect, linear, pinhole cameras. That means that distortion of the image has been removed, and projection of real-world points are linear:

$(x_l, y_l) = (f \frac{X}{Z}, f \frac{Y}{Z})$, and $(x_r, y_r) = (f \frac{(X-T_X)}{Z}, f \frac{Y}{Z})$,

where $f$ is the focal length, and $T_X$ the translation between the cameras. Please note that, since the cameras are only displaced in the $X$-axis, the image $y$-coordinates are equal (i.e., the standard stereo setup involves [horizontal epipolar lines](https://en.wikipedia.org/wiki/Epipolar_geometry)). 

The x-coordinates can be different though, leading to a non-zero _disparity_ $d$:

$d = x_l – x_r = f \frac{X}{Z} – f\frac{(X-T_X)}{Z} = f \frac{T_X}{Z}$

Matching a feature present in the left image to the same feature in the right image allows one to determine the disparity and subsequently the "depth" of the feature along the $Z$-axis. Manipulating the previous equation gives:

$Z = f \frac{T_X}{d}$

$Z$ can be determined in meters, for a calibrated camera with the focal length and displacement known. 




## The simplest stereo vision algorithm
Although the basic concept sounds easy enough, currently stereo vision is still a very active research area. There are multiple challenges to stereo vision, and there is an ongoing effort to make stereo vision algorithms more accurate and computationally more efficient. If you are interested in some of the latest algorithms and results, please see popular benchmarks such as [Middlebury](http://vision.middlebury.edu/stereo/eval3/) and [KITTI](http://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=stereo). 

In this Jupyter notebook, we will first focus on developing some intuition for what challenges stereo vision involves. To do that, we start with the simplest possible algorithm. We apply our algorithm to two stereo images obtained with the 4-gram stereo vision system that we used to make the 20-gram flapping wing robot _DelFly Explorer_ fly completely by itself.

<img src="DelFly_Explorer.jpg" width="300" alt="DelFly Explorer"/>
<center><small>The _DelFly Explorer_, described in De Wagter, C. et al. Autonomous flight of a 20-gram flapping wing mav with a 4-gram onboard stereo vision system. In the 2014 IEEE International Conference on Robotics and Automation (ICRA),  (pp. 4982-4987).</small></center>

<img alt="Stereo image from DelFly Explorer" src=" stereo_image_explorer.bmp"/>
<center><small>Stereo images we will be working with.</small></center>

Now, let us first import some necessary libraries and the function that will read the stereo image and separate the left and right image.



In [ ]:
%matplotlib inline
import numpy as np
import cv2
from matplotlib import pyplot as plt

def read_stereo_image(im="stereo_image_explorer.bmp"):
    cv_im = cv2.imread(im);
    imgL = cv_im[0:96, 126:252, :];
    imgR = cv_im[0:96, 0:126, :];
    return [imgL, imgR]


In [ ]:
# read the image:
[imgL, imgR] = read_stereo_image();

We have now read the left and right image. The following code will show them in separate windows. Open these windows and press a key to have them closed.

In [ ]:
# show the images
cv2.imshow('left', imgL);
cv2.imshow('right', imgR);

# wait for a key to be pressed before moving on:
cv2.waitKey();
cv2.destroyAllWindows();

Now it is time for the simplest stereo vision algorithm. For each pixel in the left image, at location $(x, y)$, search the corresponding pixel in the right image by look at the image intensity of a single pixel. So, the pixel intensity in the left image is $I_l(x,y)$. The algorithm then starts at $(x,y)$ in the right image and then moves more and more to the left. In other words, the algorithm looks at the intensity in the right image $I_r(x-d, y)$, for the disparity $d$ in a given fixed range $[0, D]$. The absolute difference between the pixel intensities, $|I_l(x,y) - I_r(x-d, y)|$ is stored for all disparities $d$, and the dispariry resulting in the minimal absolute difference is selected as the correct disparity. This disparity is stored in the disparity image.

<small>Small question: why does the algorithm not also have to check for coordinates to the _right_ of $(x,y)$, i.e., $(x+i,y)$, for various $i$?</small>

In [ ]:
def simple_stereo(imgL, imgR, max_disparity=30):
    
    W = imgL.shape[1];
    H = imgL.shape[0];
    
    # create the disparities image:
    Disparities = np.zeros([H, W]);
    
    # loop over the image
    for x in range(W):
        
        # in the left border of the left image, not all disparities can be investigated:
        max_disp = np.min([x, max_disparity]);
        disps = np.arange(0, max_disp+1);
        
        for y in range(H):
            # we can determine the differences in one go:
            differences = np.abs(imgL[y,x,0] - imgR[y, x-max_disp:x+1,0]);
            # the minimal difference determines the disparity
            disp_ind = np.argmin(differences);
            disparity = disps[disp_ind];
            Disparities[y, x] = disparity;
    
    return Disparities;


If you have familiarized yourself with the above code, please run it to see what the effect is.

In [ ]:
D = simple_stereo(imgL, imgR, max_disparity = 20);

plt.figure();
plt.imshow(D, cmap='hot');
plt.colorbar();
plt.draw()

Bright, yellow areas are close by, while dark areas are far away. There are many areas in the image that have a wrong disparity in the image. There are many reasons for this suboptimal result. One reason may be that 30 is quite a large value for the maximum disparity, enhancing the probability that there is a similar pixel for a 'wrong' disparity value. Please change the maximum disparity in the cell above and rerun the script to see if you can get a better result.

One can also get insight into the errors by investigating what the differences array looks like from which the best disparity is chosen. The following code will allow us to select an image coordinate $(x,y)$ in the left image, and show the according differences array.

In [ ]:
def get_differences_curve(imgL, imgR, x, y, max_disparity=30):
    
    # determine the disparities that will be investigated:
    max_disp = np.min([x, max_disparity]);
    disps = np.arange(0, max_disp+1);
    
    # we can determine the differences in one go:
    differences = np.abs(imgL[y,x,0] - imgR[y, x-max_disp:x+1,0]);
    # the minimal difference determines the disparity
    disp_ind = np.argmin(differences);
    disparity = disps[disp_ind];
    
    return [differences, disps, disp_ind];

In [ ]:
print('Image size, width = {}, height = {}'.format(imgL.shape[1], imgL.shape[0]))
[differences, disps, disp_ind] = get_differences_curve(imgL, imgR, 48, 64); 
plt.figure();
plt.plot(disps, differences);
plt.plot(disps[disp_ind], differences[disp_ind], 'x', markersize=10);
plt.draw();

This first coordinate already shows a big problem, which is the existence of multiple minima. Try changing the coordinate to find other apparent problems for the automatic selection of the best disparity. Please stay in bounds of the image - the code is not that smart... We do print the size of the image above: $126 \times 96$ pixels. Note that Python's indexing starts at 0 and that $(0,0)$ is the top left of the image. 

## Window-based matching
A major reason for the wrong results above is that a single pixel is a very poor basis for matching between images. A given pixel value may occur in many different regions of the image, also on a single scan line. The next simplest stereo vision algorithm acknowledges this fact, and hence focuses on the differences between pixels in a window centered on $(x,y)$ and $(x-d, y)$. Now, we do not just take the absolute difference in intensity at a single pixel, but in the entire window:

SAD $= \sum_{i=-C}^C \sum_{j=-C}^C \left| I_l(x+i, y+j) - I_r(x+i-d, y+j) \right| $,

where SAD stands for the Sum of Absolute Differences, and serves as the new 'difference' value, and the window size is $2C+1$. 

The algorithm is given below:

In [ ]:
def windowed_stereo(imgL, imgR, max_disparity=30, window_half_size=3):
    
    W = imgL.shape[1];
    H = imgL.shape[0];
    
    # create the disparities image:
    Disparities = np.zeros([H, W]);
    
    # loop over the image
    for x in np.arange(window_half_size, W-window_half_size):
        
        # in the left border of the left image, not all disparities can be investigated:
        max_disp = np.min([x-window_half_size, max_disparity]);
        if(max_disp >= 0):
            disps = np.arange(0, max_disp+1);
            differences = np.zeros([len(disps), 1]);
            
            for y in np.arange(window_half_size, H-window_half_size):
                
                window_left = imgL[y-window_half_size:y+window_half_size, x-window_half_size:x+window_half_size, 0];
                
                for d in disps:
                    window_right = imgR[y-window_half_size:y+window_half_size, x-d-window_half_size:x-d+window_half_size, 0];
                    differences[d] = np.sum(np.abs(window_left.astype(float) - window_right.astype(float)));
                
                # the minimal difference determines the disparity
                disp_ind = np.argmin(differences);
                disparity = disps[disp_ind];
                Disparities[y, x] = disparity;
    
    return Disparities;

Now, let's run this algorithm with a max disparity of 30 and half window size $C=3$:

In [ ]:
D = windowed_stereo(imgL, imgR, max_disparity=30, window_half_size=3);
plt.figure();
plt.imshow(D, cmap='hot');
plt.colorbar();
plt.draw()


This already looks much better! Try to play with the maximum disparity and especially with the window size to see the effects on the disparity map. Can you get much better results? What happens when the half window size becomes big, like 8 for example? 

## 'Better' methods
The algorithms just shown for stereo vision are the most elementary ones that can be imagined. The goal above was to let you gain some fundamental insights into stereo vision. Both algorithms above have in common that they only look _locally_ at what the best disparity is. That is, for each coordinate $(x,y)$ they select the disparity with lowest error, without considering certain regularities in the real world. For instance, in the real world, disparities tend to change smoothly when pixels belong to the same object and then may change abruptly when there is an edge of the object. _Global_ methods take such regularities into account, i.e., they make some assumptions on how disparities change over the image and optimize the disparities over the whole image.

Stereo vision algorithms can be better in terms of accuracy, or computational effort, or both. There are is an increasing number of algorithms that satisfies both criteria quite well. A well-known algorithm that is quite efficient and accurate is "Semi-global block matching" - _SGBM_, by Hirschmuller, H. (2008). Stereo processing by semiglobal matching and mutual information. IEEE Transactions on pattern analysis and machine intelligence, 30(2), 328-341.

SGBM is part of openCV and easy to implement in Python:

In [ ]:
def calculate_disparities(imgL, imgR, window_size=7, min_disp=0, num_disp=16):
    # semi-global matching:
    stereo = cv2.StereoSGBM_create(numDisparities = num_disp, blockSize = window_size);
    disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0;
    return disp; 

Let us run it on our images:

In [ ]:
D = calculate_disparities(imgL, imgR, window_size=7, min_disp=0, num_disp=16)
plt.figure();
plt.imshow(D, cmap='hot');
plt.colorbar();
plt.draw()

This looks good!

Perhaps you can play with the parameters to get a better result than I did. Watch out: the num_disp parameter apparently has to be a multiple of 16...

Unfortunately, on the DelFly we only had a processor with 168 MHz and 192 kB of memory... Hence, we to make a computationally cheaper method...